In [ ]:
%%capture
!pip install datasets torchaudio jiwer librosa evaluate numpy
!pip install transformers==4.11.13

In [ ]:
import datasets
from datasets import load_dataset
import os
import evaluate
import numpy as np
os.environ["HF_TOKEN"] = "Add your own Hugging Face Token here"

common_voice_train = load_dataset("mozilla-foundation/common_voice_17_0", "nl", split="train", trust_remote_code=True)
common_voice_test = load_dataset("mozilla-foundation/common_voice_17_0", "nl", split="test", trust_remote_code=True)

common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/12.7k [00:00<?, ?B/s]

common_voice_17_0.py:   0%|          | 0.00/8.19k [00:00<?, ?B/s]

languages.py:   0%|          | 0.00/3.92k [00:00<?, ?B/s]

release_stats.py:   0%|          | 0.00/132k [00:00<?, ?B/s]

...


Reading metadata...: 2771it [00:00, 124863.47it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 5580it [00:00, 100046.24it/s]


Generating validated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 14545it [00:00, 145443.55it/s]
Reading metadata...: 29135it [00:00, 145704.97it/s]
Reading metadata...: 43706it [00:00, 140005.52it/s]
Reading metadata...: 57732it [00:00, 139450.83it/s]
Reading metadata...: 71692it [00:00, 132459.18it/s]
Reading metadata...: 90449it [00:00, 134184.79it/s]


In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))


In [ ]:
show_random_elements(common_voice_train.remove_columns(["path", "audio"]), num_examples=10)

,sentence,variant
0,"Niet op alle punten zijn we tevreden, maar we zijn een eind opgeschoten.",
1,Betreffende de aanvullende en gewijzigde begrotingen wil ik slechts drie opmerkingen maken.,
2,Mensen hebben recht op een antwoord op dergelijke vragen.,
3,Dat is eveneens een belangrijke doorbraak.,
4,Het is een beleid dat langzamerhand tot wanhoop stemt.,
5,Mijn derde punt vormt de kern van het democratische debat.,
6,De wereld is in feite machteloos.,
7,Ook in dit verband moeten we ons voorbereiden op een ernstig democratieprobleem.,
8,Geen van de eisers was dus aanwezig.,
9,Werkloosheid voorkomen is nu eenmaal menselijker en bovendien veel goedkoper.,


In [ ]:
import re
chars_to_ignore_regex = r'[\,\?\.\!\-\;\:\"\“\%\‘\”\�€—]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower() + " "
    return batch

In [ ]:
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)

Map:   0%|          | 0/34898 [00:00<?, ? examples/s]

Map:   0%|          | 0/11266 [00:00<?, ? examples/s]

In [ ]:
show_random_elements(common_voice_train.remove_columns(["path","audio"]))

,sentence,variant
0,in hun wanhoop gaan zij soms over tot gewelddaden en zelfs tot zelfdoding,
1,de millenniumdoelstellingen zijn primair een morele kwestie,
2,soms komt dit omdat ze economisch onvoldoende waard zijn,
3,verzoening is bijna onvoorstelbaar tenzij er een rechtvaardig nieuw systeem wordt ingesteld,
4,zij willen dat de euro nog sterker wordt,
5,dat betekent dat we op twee fronten tegelijk actief moeten zijn,
6,tegelijkertijd willen wij op meer zaken aandringen,
7,de tekst moet dan ook kort en eenvoudig zijn maar niet banaal,
8,we moeten ons dus afvragen wat nietsdoen kost,
9,ik wens u tot slot toch veel geluk het komende halfjaar,


In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

Map:   0%|          | 0/34898 [00:00<?, ? examples/s]

Map:   0%|          | 0/11266 [00:00<?, ? examples/s]

In [ ]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [ ]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'l': 0,
 'é': 1,
 'k': 2,
 'r': 3,
 'đ': 4,
 'ä': 5,
 'ü': 6,
 '´': 7,
 'h': 8,
 'n': 9,
 'ó': 10,
 'ö': 11,
 'q': 12,
 'ţ': 13,
 'o': 14,
 't': 15,
 'w': 16,
 'ë': 17,
 '’': 18,
 'c': 19,
 '–': 20,
 'î': 21,
 'f': 22,
 'ć': 23,
 'x': 24,
 'v': 25,
 'z': 26,
 '(': 27,
 '=': 28,
 'û': 29,
 'ï': 30,
 'j': 31,
 'ú': 32,
 'á': 33,
 's': 34,
 '…': 35,
 'ç': 36,
 ')': 37,
 'b': 38,
 'ê': 39,
 'i': 40,
 'à': 41,
 'm': 42,
 'y': 43,
 ' ': 44,
 'g': 45,
 "'": 46,
 'd': 47,
 'š': 48,
 'ă': 49,
 '&': 50,
 'a': 51,
 'í': 52,
 'e': 53,
 'p': 54,
 'u': 55,
 'è': 56}

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

59

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
processor.save_pretrained("/content/gdrive/MyDrive/DutchOnlyModel")

[]

In [ ]:
common_voice_train[0]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/7505314af06ede8652d9a45debf2013eed934b975e042f7cdd9e72b74740a86f/nl_train_0/common_voice_nl_22526031.mp3',
 'array': array([ 0.00000000e+00, -4.67330100e-14,  8.23077500e-15, ...,
        -9.62244667e-06, -9.05787874e-06,  1.13442979e-06]),
 'sampling_rate': 48000}

In [ ]:
from datasets import Audio
import torchaudio

common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))

In [ ]:
common_voice_train[0]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/7505314af06ede8652d9a45debf2013eed934b975e042f7cdd9e72b74740a86f/nl_train_0/common_voice_nl_22526031.mp3',
 'array': array([-3.41060513e-12, -1.36424205e-12, -5.68434189e-12, ...,
        -4.97980363e-06,  6.58454155e-07, -5.71697092e-06]),
 'sampling_rate': 16000}

In [ ]:
import random
import IPython.display as ipd
from IPython.core.display import display, HTML

def show_random_sentence_with_audio(dataset):
    pick = random.randint(0, len(dataset) - 1)  # Pick one random index

    sentence = dataset[pick]["sentence"]
    audio_data = dataset[pick]["audio"]["array"]
    sample_rate = dataset[pick]["audio"]["sampling_rate"]

    # Convert the audio data into an embedded player
    audio_html = ipd.Audio(data=audio_data, rate=sample_rate)._repr_html_()

    # Generate HTML output
    html = f"""
    <p><strong>Sentence:</strong> {sentence}</p>
    {audio_html}
    """

    display(HTML(html))

# Call the function with your dataset
show_random_sentence_with_audio(common_voice_train)

In [ ]:
rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]["sentence"])
print("Input array shape:", common_voice_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", common_voice_train[rand_int]["audio"]["sampling_rate"])

Target text: als we het probleem niet begrijpen zullen we de oplossing ook niet begrijpen 
Input array shape: (83712,)
Sampling rate: 16000


In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]

    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [ ]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names, num_proc=4)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names, num_proc=4)

Map (num_proc=4):   0%|          | 0/34898 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call

Map (num_proc=4):   0%|          | 0/11266 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
wer_metric = evaluate.load("wer")

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53",
    attention_dropout=0.05,  # Lower dropout due to smaller dataset
    hidden_dropout=0.05,  # Reduce overfitting risk
    feat_proj_dropout=0.0,  # Keep feature projection dropout at 0
    mask_time_prob=0.08,  # Increase slightly for better generalization
    layerdrop=0.1,  # Keep default, balances stability & regularization
    ctc_loss_reduction="mean",  # Standard setting
    pad_token_id=processor.tokenizer.pad_token_id,  # Correct padding
    vocab_size=len(processor.tokenizer)  # Ensure correct vocabulary size
)

NameError: name 'processor' is not defined

In [ ]:
model.freeze_feature_extractor()

/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:2176: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [ ]:
model.gradient_checkpointing_enable()

In [ ]:
from transformers import TrainingArguments, Trainer # import TrainingArguments and Trainer

training_args = TrainingArguments(
  output_dir="/content/gdrive/MyDrive/Dutch_Only_Model",
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  fp16=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=10,
  learning_rate=2e-4,
  warmup_steps=1000,
  save_total_limit=2,
  max_steps=6000  # <-- Stops at 6000 steps
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

NameError: name 'model' is not defined

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: izabelle (izabelle-universiteit-utrecht) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
500,3.024800,3.004823,1.000000
1000,0.651700,0.382024,0.449367
1500,0.409000,0.258959,0.366053
2000,0.360500,0.230276,0.324937
2500,0.336700,0.202489,0.308197
3000,0.297100,0.188400,0.297143
3500,0.325200,0.177636,0.287159
4000,0.188100,0.170264,0.281358
4500,0.210200,0.162995,0.271942
5000,0.180800,0.156356,0.267359


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in 

TrainOutput(global_step=6000, training_loss=1.0104958010315894, metrics={'train_runtime': 55137.7326, 'train_samples_per_second': 3.482, 'train_steps_per_second': 0.109, 'total_flos': 3.398542027021394e+19, 'train_loss': 1.0104958010315894, 'epoch': 1.6295491580662684})

In [ ]:
model.save_pretrained("./Dutch_Only_Model")
processor.save_pretrained("./Dutch_Only_Processor")

NameError: name 'model' is not defined